In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
get_index = []
get_value = []

In [14]:
def get_order(m_equal,stop,count = 0):                      
    """
    获得隶属度大小排序与对应索引
    """
    c = np.triu(m_equal) - np.diag(np.diag(m_equal)) #对称函数只需考虑倒三角

    i, j = argmax(c)
    get_index.extend([[i, j]])
    get_value.append(c[i, j])
    c[i,j] = 0                                      #将已取元素设为最小值，再次获取元素
    
    count += 1                                      #判断迭代是否继续进行
    if count < stop:
        return get_order(c,stop,count)
    else:
        return get_index, get_value

In [ ]:
    
#     temp1 = np.sort(c, axis = 0)
#     temp2 = np.argsort(c, axis = 0)
#     temp3 = np.sort(temp1)
#     temp4 = np.argsort(temp1)                               #对元素及其索引按大小顺序进行排列

#     a = temp4[-1, -1]
#     b = temp2[-1, a]                                        #对角阵只需考虑行号小于列号的元素
#     i = min(a, b)
#     j = max(a, b)

In [15]:
def index_value(m_equal,n):
    """
    获得各个属性对应的、第一次应该被聚为一类的隶属度
    """
    stop = np.cumsum(np.arange(n))[-1]                  #计算循环次数
    get_index, get_value = get_order(m_equal, stop)     #获得隶属度大小关系与对应索引

    value = list(set(get_value))                       #将元素重复值删除并排序
    value.sort(reverse = True)

    index = [[] for i in range(len(value))]            #按排序后的元素值获取其对应的属性号
    for i in range(len(value)):
        for j in range(len(get_index)):
            if get_value[j] == value[i]:
                index[i] += get_index[j]
                index[i] = list(set(index[i]))
                index[i].sort()

    return index, value

In [16]:
def get_index_rev(index):
    """
    由于列表是可变对象，需要不断变化获得列表的逆序
    """
    index_rev = []
    index.reverse()
    for x in index:
        index_rev.append(x)
    index.reverse()
    return index_rev

In [17]:
def get_xticks(m_equal,n,ix):
    index, value = index_value(m_equal,n)
    index_rev = get_index_rev(index)
    xticks = [index_rev[0]]                             #聚类图反过来就是分类图，根据分类的思想，将较少的元素或较小
                                                        #的树放在右边，才能保证画图时不出现交叉，得到各属性出现的顺序
    for i in range(1,len(index_rev)):
        flag = False
        for j in range(len(xticks)):
            if set(index_rev[i]) <= set(xticks[j]):
                x = list(set(xticks[j]) - set(index_rev[i]))
                if x != []:
                    if len(x) < len(index_rev[i]):
                        xticks.insert(j+1, x)
                        xticks[j] = index_rev[i]
                    else:
                        xticks.insert(j+1, index_rev[i])
                        xticks[j] = x
                flag = True
                break
        if flag == False:
            xticks.append(index_rev[i])
    
    xticks_label = []                            #将排列好的属性与画图时的真实坐标生成一个字典
    for x in xticks:
        xticks_label += x
    
    
    dict = {}
    for keys, values in zip(xticks_label,ix):
        dict[keys] = values

    return xticks_label, dict

In [18]:
def draw(m_equal,n):

    x = []
    y = []
    ix = list(range(39))

    index, value = index_value(m_equal,n)
    print('Done')
    index_rev = get_index_rev(index)
    print('Done')
    xticks_label, dict = get_xticks(m_equal,n,ix)
    print('Done')
    
    fig = plt.figure(figsize = (12,6))
    ax = fig.add_subplot(111)
    
    point_y = []
    for i in xticks_label:
        for j in range(len(index)):
            if i in index[j]:
                x.append(dict[i])
                y.append(1)
                x.append(dict[i])
                y.append(value[j])
                point_y.append(value[j])
                ax.plot(x, y)
                x = []
                y = []
                break


    a = ax.xaxis.set_ticks_position('top')
    a = ax.set_xticklabels(xticks_label)
    a = ax.set_xticks(ix)
    plt.show()